In [1]:
import numpy as np
import optimal_estimation as oe

%load_ext autoreload
%autoreload 2

Define 'forward' function. This can be *any* kind of python function (e.g. LUTs), as long as it accepts x, params and returns y.

In [2]:
def func(x,*params):
    '''
    simple example of a function R^2 --> R^3
    (two state variables and three measurements)

    In general every function is possible but it must  accept: 
    Input:
           x:  n-element np-array 
      params:  additional parameter,that are needed (e.g. geometry ..)
               could be anything tupel,dict,list,func ....
    Output:
           y:  m-element  np-array

    '''
    # this is just a quick and dirty check
    # if params have been used
    try:
        dum=1.**params[0]
        exp=params[0]
    except:
        exp=1.
    # simple non-linear useless function    
    return np.asarray([ 13 + 6*x[0]**exp + 4*x[1] + 7*np.log(x[0]*x[1])
                          , 2 - 3*x[0] + 2*x[1]**exp +   np.sqrt(x[0])*np.log(x[1])
                          ,       x[0] - 5*x[1] -   np.sqrt(x[0]*x[1])**exp
                          ])
    # simple linear useless function    
    #return np.asarray([ 13 + 6*x[0] + 4*x[1]
    #                      , 2 - 3*x[0] + 2*x[1]  
    #                      ,       x[0] - 5*x[1] 
    #                      ])

Test forward

In [3]:
x=np.array([1.9,2.8])
print func(x)
print func(x,1.8)

[ 47.30031312   3.31923242 -14.40651252]
[ 54.95087249  10.48110641 -16.60111873]


Define 'forward' function Jacobian. This can be *any* kind of python function (e.g. LUTs), as long as it accepts x, params and returns an np-array of dimension (y.size,x.size)   

In [4]:
def func_jaco(x,*params):
    '''
    simple example of a jacobian for a function R^2 --> R^3
    (two state variables and three measurements)

    In general every kind of function is possible, but it must accept: 
    Input:
           x:  n-element np-array 
      params:  additional parameter,that are needed (e.g. geometry ..)
               could be anything tupel,dict,list,func ....
    Output:
           j:  (size(y),size(x))  np-array (here: 3,2)

    '''
    # this is just a quick and dirty check
    # if params have been used
    try:
        dum=1.**params[0]
        exp=params[0]
    except:
        exp=1.
    ny=3
    nx=2
    dx=np.array([0.001,0.001])
    jac=np.zeros((ny,nx))  # zeilen zuerst, spalten später!!!!!!!
    for ix in range(nx):
        dxm=x*1.
        dxp=x*1.
        dxm[ix]=dxm[ix]-dx[ix]
        dxp[ix]=dxp[ix]+dx[ix]
        dyy=func(dxp,exp)-func(dxm,exp)
        for iy in range(ny):
            jac[iy,ix]=dyy[iy]/dx[ix]/2.
    return jac

In [5]:
print func_jaco(x)
print func_jaco(x,1.2)

[[ 9.68421087  6.50000011]
 [-2.62651777  2.49228748]
 [ 0.393023   -5.41187724]]
[[ 11.87042713   6.50000011]
 [ -2.62651777   3.4410707 ]
 [  0.13911599  -5.58417128]]


Generate inverse function

In [6]:
# lower limit for x
a=np.array([-3.,0.])
# upper limit for x
b=np.array([53.,10.])

#this creates the inverse function
inv_func=oe.my_inverter(func,a,b,jaco=func_jaco)

Test inverse function  with *pure* Newton

In [7]:
#measurement
y=np.array([ 47.30031312 ,  3.31923242, -14.40651252])

#Inversion retrieves x (=state) 
print inv_func(y,method=0,full=True)
print inv_func(func([3.4,5.7]),method=0)

result(x=array([ 1.9,  2.8]), j=array([[ 9.68421087,  6.50000011],
       [-2.62651777,  2.49228748],
       [ 0.393023  , -5.41187724]]), conv=True, ni=5, g=array([[ 0.08176003, -0.06936117,  0.06625653],
       [ 0.02652773,  0.08047365, -0.11585745]]), a=array([[ 1.,  0.],
       [ 0.,  1.]]), sr=array([[ 0.,  0.],
       [ 0.,  0.]]), cost=2.1421177383297344e-17)
[ 3.4  5.7]


Test inverse with Newton and measurement error covariance

In [8]:
#measurement error covariance
se=np.array([[1,0.,0],[0,1.,0],[0,0,10.]])


# y is measurement with error at y[2]
y=func(np.array([1.9,2.8]))+np.array([0.,0.,2.])

#Inversion
print inv_func(y,se=se,method=1,full=True)
print inv_func(y,se=se,method=1,full=True,jaco=None)
print inv_func(y,se=se,method=1)

result(x=array([ 1.93199344,  2.74437967]), j=array([[ 9.62320104,  6.5506676 ],
       [-2.63684067,  2.50647574],
       [ 0.40407817, -5.41951813]]), conv=True, ni=5, g=array([[ 0.06569559, -0.13702737,  0.01603356],
       [ 0.05482181,  0.19583083, -0.02768425]]), a=array([[ 1.,  0.],
       [ 0.,  1.]]), sr=array([[ 0.02566316, -0.0276714 ],
       [-0.0276714 ,  0.04901932]]), cost=0.33701632767915607)
result(x=array([ 1.93199476,  2.74437827]), j=array([[ 9.62421344,  6.55068008],
       [-2.63680284,  2.50647839],
       [ 0.40401594, -5.41951907]]), conv=True, ni=5, g=array([[ 0.06569181, -0.13701832,  0.01603327],
       [ 0.05481727,  0.19583852, -0.02768596]]), a=array([[ 1.,  0.],
       [ 0.,  1.]]), sr=array([[ 0.02566009, -0.02767138],
       [-0.02767138,  0.04902278]]), cost=0.33701632774755708)
[ 1.93199344  2.74437967]


Test inverse with optimal estimation. Needed quantities are:  measurment error covariance, apriori error covariance, apriori and first guess

In [9]:
#measurement error covariance
se=np.array([[1,0.,0],[0,1.,0],[0,0,10.]])
#apriori error covariance
sa=np.array([[3,0.],[0,10.]])
#apriori
xa=np.array([2.,3.])
#first guess
fg=np.array([2.4,8.])
# y is measurement with error at y[2]
y=func(np.array([1.9,2.8]))+np.array([0.,0.,2.])

#Inversion
print inv_func(y,se=se,sa=sa,xa=xa,fg=fg,method=2,full=True)
print inv_func(y,se=se,sa=sa,xa=xa,fg=fg,method=2,full=False)

result(x=array([ 1.93187239,  2.74499846]), j=array([[ 9.62342807,  6.55009262],
       [-2.63674819,  2.50634571],
       [ 0.40399232, -5.41945771]]), conv=True, ni=4, g=array([[ 0.065291  , -0.13532882,  0.01582276],
       [ 0.05515425,  0.19364717, -0.02740756]]), a=array([[ 0.9915435 ,  0.0027305 ],
       [ 0.00910168,  0.99514629]]), sr=array([[ 0.02536949, -0.02730503],
       [-0.02730503,  0.04853707]]), cost=0.34507906118559811)
[ 1.93187239  2.74499846]


In [10]:
#using fparams
param=1.3
# y is measurement with error at y[2]
y=func([1.9,2.8],param)+np.array([0.,0.,2.])
print inv_func(y,se=se,sa=sa,xa=xa,fg=fg,method=2,fparams=param,jparams=param,full=True)
print inv_func(y,se=se,sa=sa,xa=xa,fg=fg,method=2,fparams=param,jparams=param,full=False,jaco=None)

result(x=array([ 1.9155322 ,  2.76319171]), j=array([[  1.31337500e+01,   6.53330245e+00],
       [ -2.63281566e+00,   4.02780851e+00],
       [ -2.37266988e-03,  -5.69487654e+00]]), conv=True, ni=4, g=array([[ 0.05894822, -0.0843424 ,  0.0077355 ],
       [ 0.03373762,  0.16659581, -0.01849345]]), a=array([[ 0.99625082,  0.0013588 ],
       [ 0.00452932,  0.99675205]]), sr=array([[ 0.01124754, -0.01358796],
       [-0.01358796,  0.03247951]]), cost=0.36570303461665754)
[ 1.91553245  2.76319141]


In [11]:
#Diagnostics took time!
%timeit dum=inv_func(y,se=se,sa=sa,xa=xa,fg=fg,method=2,fparams=param,jparams=param,full=True)
%timeit dum=inv_func(y,se=se,sa=sa,xa=xa,fg=fg,method=2,fparams=param,jparams=param,full='fast')
%timeit dum=inv_func(y,se=se,sa=sa,xa=xa,fg=fg,method=2,fparams=param,jparams=param,full=False)

1000 loops, best of 3: 473 µs per loop
1000 loops, best of 3: 411 µs per loop
1000 loops, best of 3: 378 µs per loop


In [12]:
%timeit numpy.asarray([1,2.,4.])
%timeit numpy.array([1,2.,4.])

10000 loops, best of 3: 19.9 µs per loop
100000 loops, best of 3: 16.6 µs per loop
